# Section Computing Pi: Compute Pi with OpenMP


Adapted from: [https://github.com/gjbex/Fortran-MOOC/tree/master/source_code/computing_pi](https://github.com/gjbex/Fortran-MOOC/tree/master/source_code/computing_pi)

## This program demonstrates computing $\pi$ in Fortran using OpenMP.  The program uses the following equation for computation:

$$
\Large \pi = 4 \int_{0}^{1} \sqrt{1-x^2}dx
$$

```fortran
program compute_pi_omp
  use, intrinsic :: iso_fortran_env, only : DP => REAL64, I8 => INT64
  implicit none
  integer(kind=I8) :: i, nr_iters
  real(kind=DP) :: delta, x, pi_val

  pi_val = 0.0_DP
  nr_iters = get_nr_iters()
  delta = 1.0_DP/nr_iters
  x = 0.0_DP

  !$omp parallel do default(none) private(x) shared(delta, nr_iters) reduction(+:pi_val)
  do i = 1, nr_iters
      x = i*delta
      pi_val = pi_val + sqrt(1.0_DP - x**2)
  end do
  !$omp end parallel do

  pi_val = 4.0_DP*pi_val/nr_iters
  print '(A, I10, A, F25.15)', "After ", nr_iters, " loops, Pi = ", pi_val

contains
   
  function get_nr_iters() result(nr_iters)
      use, intrinsic :: iso_fortran_env, only : error_unit
      implicit none
      integer(kind=I8) :: nr_iters
      integer(kind=I8), parameter :: default_nr_iters = 1000_I8
      character(len=1024) :: buffer, msg
      integer :: istat

      if (command_argument_count() >= 1) then
          call get_command_argument(1, buffer)
          read (buffer, fmt=*, iostat=istat, iomsg=msg) nr_iters
          if (istat /= 0) then
              write (unit=error_unit, fmt='(2A)') &
                  'error: ', msg
              stop 1
          end if
      else
          nr_iters = default_nr_iters
      end if
  end function get_nr_iters

end program compute_pi_omp
```

The above program is compiled and run using Fortran Package Manager (fpm):

## Build the Program using FPM (Fortran Package Manager)

In [1]:
import os
root_dir = ""
root_dir = os.getcwd()

In [2]:
code_dir = root_dir + "/" + "Fortran_Code/Section_Computing_Pi_Compute_Pi_OpenMP"

In [3]:
code_app_dir = code_dir + "/" + "app"

In [4]:
os.chdir(code_app_dir)

In [5]:
%%capture
%%writefile section_computing_pi_compute_pi_openmp.f90
program compute_pi_omp
  use, intrinsic :: iso_fortran_env, only : DP => REAL64, I8 => INT64
  implicit none
  integer(kind=I8) :: i, nr_iters
  real(kind=DP) :: delta, x, pi_val

  pi_val = 0.0_DP
  nr_iters = get_nr_iters()
  delta = 1.0_DP/nr_iters
  x = 0.0_DP

  !$omp parallel do default(none) private(x) shared(delta, nr_iters) reduction(+:pi_val)
  do i = 1, nr_iters
      x = i*delta
      pi_val = pi_val + sqrt(1.0_DP - x**2)
  end do
  !$omp end parallel do

  pi_val = 4.0_DP*pi_val/nr_iters
  print '(A, I10, A, F25.15)', "After ", nr_iters, " loops, Pi = ", pi_val

contains
   
  function get_nr_iters() result(nr_iters)
      use, intrinsic :: iso_fortran_env, only : error_unit
      implicit none
      integer(kind=I8) :: nr_iters
      integer(kind=I8), parameter :: default_nr_iters = 1000_I8
      character(len=1024) :: buffer, msg
      integer :: istat

      if (command_argument_count() >= 1) then
          call get_command_argument(1, buffer)
          read (buffer, fmt=*, iostat=istat, iomsg=msg) nr_iters
          if (istat /= 0) then
              write (unit=error_unit, fmt='(2A)') &
                  'error: ', msg
              stop 1
          end if
      else
          nr_iters = default_nr_iters
      end if
  end function get_nr_iters

end program compute_pi_omp

In [6]:
!bat *.f90

───────┬────────────────────────────────────────────────────────────────────────
       │ File: section_computing_pi_compute_pi_openmp.f90
───────┼────────────────────────────────────────────────────────────────────────
   1   │ program compute_pi_omp
   2   │   use, intrinsic :: iso_fortran_env, only : DP => REAL64, I8 => INT64
   3   │   implicit none
   4   │   integer(kind=I8) :: i, nr_iters
   5   │   real(kind=DP) :: delta, x, pi_val
   6   │ 
   7   │   pi_val = 0.0_DP
   8   │   nr_iters = get_nr_iters()
   9   │   delta = 1.0_DP/nr_iters
  10   │   x = 0.0_DP
  11   │ 
  12   │   !$omp parallel do default(none) private(x) shared(delta, nr_iters) re
       │ duction(+:pi_val)
  13   │   do i = 1, nr_iters
  14   │       x = i*delta
  15   │       pi_val = pi_val + sqrt(1.0_DP - x**2)
  16   │   end do
  17   │   !$omp end parallel do
  18   │ 
  19   │   pi_val = 4.0_DP*pi_val/nr_iters
  20   │   print '(A, I10, A, F25.15)', "After ", nr_iters, " loops, Pi = ", pi_
       │ val

In [7]:
os.chdir(code_dir)

Since this program uses the OpenMP external library, the fpm.toml file had to be augmented as below.

In [8]:
!bat fpm.toml

───────┬────────────────────────────────────────────────────────────────────────
       │ File: fpm.toml
───────┼────────────────────────────────────────────────────────────────────────
   1   │ name = "Section_Computing_Pi_Compute_Pi_OpenMP"
   2   │ 
   3   │ 
   4   │ [build]
   5   │ auto-executables = true
   6   │ auto-tests = true
   7   │ auto-examples = true
   8   │ external-modules = "openmp"
   9   │ 
  10   │ [install]
  11   │ library = false
  12   │ 
  13   │ [[executable]]
  14   │ name="Section_Computing_Pi_Compute_Pi_OpenMP"
  15   │ source-dir="app"
  16   │ main="section_computing_pi_compute_pi_openmp.f90"
───────┴────────────────────────────────────────────────────────────────────────


In [9]:
build_status = os.system("fpm build 2>/dev/null")

## Run the Program using FPM (Fortran Package Manager)

In [18]:
%%timeit -r 10 -n 1
exec_status = os.system("fpm run -- 10 2>/dev/null")

After         10 loops, Pi =         2.904518326248318
After         10 loops, Pi =         2.904518326248318
After         10 loops, Pi =         2.904518326248318
After         10 loops, Pi =         2.904518326248318
After         10 loops, Pi =         2.904518326248318
After         10 loops, Pi =         2.904518326248318
After         10 loops, Pi =         2.904518326248318
After         10 loops, Pi =         2.904518326248318
After         10 loops, Pi =         2.904518326248318
After         10 loops, Pi =         2.904518326248318
20.1 ms ± 6.85 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [23]:
%%timeit -r 10 -n 1
exec_status = os.system("fpm run -- 100 2>/dev/null")

After        100 loops, Pi =         3.120417031779045
After        100 loops, Pi =         3.120417031779045
After        100 loops, Pi =         3.120417031779045
After        100 loops, Pi =         3.120417031779045
After        100 loops, Pi =         3.120417031779045
After        100 loops, Pi =         3.120417031779045
After        100 loops, Pi =         3.120417031779045
After        100 loops, Pi =         3.120417031779045
After        100 loops, Pi =         3.120417031779045
After        100 loops, Pi =         3.120417031779045
24.3 ms ± 6.18 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [24]:
%%timeit -r 10 -n 1
exec_status = os.system("fpm run -- 1000 2>/dev/null")

After       1000 loops, Pi =         3.139555466911023
After       1000 loops, Pi =         3.139555466911023
After       1000 loops, Pi =         3.139555466911023
After       1000 loops, Pi =         3.139555466911023
After       1000 loops, Pi =         3.139555466911023
After       1000 loops, Pi =         3.139555466911023
After       1000 loops, Pi =         3.139555466911023
After       1000 loops, Pi =         3.139555466911023
After       1000 loops, Pi =         3.139555466911023
After       1000 loops, Pi =         3.139555466911023
The slowest run took 5.24 times longer than the fastest. This could mean that an intermediate result is being cached.
30.1 ms ± 16.7 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [26]:
%%timeit -r 10 -n 1
exec_status = os.system("fpm run -- 10000 2>/dev/null")

After      10000 loops, Pi =         3.141391477611321
After      10000 loops, Pi =         3.141391477611321
After      10000 loops, Pi =         3.141391477611321
After      10000 loops, Pi =         3.141391477611321
After      10000 loops, Pi =         3.141391477611321
After      10000 loops, Pi =         3.141391477611321
After      10000 loops, Pi =         3.141391477611321
After      10000 loops, Pi =         3.141391477611321
After      10000 loops, Pi =         3.141391477611321
After      10000 loops, Pi =         3.141391477611321
23.5 ms ± 6.29 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [30]:
%%timeit -r 10 -n 1
exec_status = os.system("fpm run -- 100000 2>/dev/null")

After     100000 loops, Pi =         3.141572616402007
After     100000 loops, Pi =         3.141572616402007
After     100000 loops, Pi =         3.141572616402007
After     100000 loops, Pi =         3.141572616402007
After     100000 loops, Pi =         3.141572616402007
After     100000 loops, Pi =         3.141572616402007
After     100000 loops, Pi =         3.141572616402007
After     100000 loops, Pi =         3.141572616402007
After     100000 loops, Pi =         3.141572616402007
After     100000 loops, Pi =         3.141572616402007
25.9 ms ± 11.7 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [31]:
%%timeit -r 10 -n 1
exec_status = os.system("fpm run -- 1000000 2>/dev/null")

After    1000000 loops, Pi =         3.141590652413977
After    1000000 loops, Pi =         3.141590652413977
After    1000000 loops, Pi =         3.141590652413977
After    1000000 loops, Pi =         3.141590652413977
After    1000000 loops, Pi =         3.141590652413977
After    1000000 loops, Pi =         3.141590652413977
After    1000000 loops, Pi =         3.141590652413977
After    1000000 loops, Pi =         3.141590652413977
After    1000000 loops, Pi =         3.141590652413977
After    1000000 loops, Pi =         3.141590652413977
The slowest run took 6.39 times longer than the fastest. This could mean that an intermediate result is being cached.
33.6 ms ± 23.4 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [16]:
%%timeit -r 10 -n 1
exec_status = os.system("fpm run -- 10000000 2>/dev/null")

After   10000000 loops, Pi =         3.141592453552712
After   10000000 loops, Pi =         3.141592453552712
After   10000000 loops, Pi =         3.141592453552712
After   10000000 loops, Pi =         3.141592453552712
After   10000000 loops, Pi =         3.141592453552712
After   10000000 loops, Pi =         3.141592453552712
After   10000000 loops, Pi =         3.141592453552712
After   10000000 loops, Pi =         3.141592453552712
After   10000000 loops, Pi =         3.141592453552712
After   10000000 loops, Pi =         3.141592453552712
48.8 ms ± 12.7 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [32]:
%%timeit -r 10 -n 1
exec_status = os.system("fpm run -- 100000000 2>/dev/null")

After  100000000 loops, Pi =         3.141592633586875
After  100000000 loops, Pi =         3.141592633586875
After  100000000 loops, Pi =         3.141592633586875
After  100000000 loops, Pi =         3.141592633586875
After  100000000 loops, Pi =         3.141592633586875
After  100000000 loops, Pi =         3.141592633586875
After  100000000 loops, Pi =         3.141592633586875
After  100000000 loops, Pi =         3.141592633586875
After  100000000 loops, Pi =         3.141592633586875
After  100000000 loops, Pi =         3.141592633586875
254 ms ± 20.2 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [34]:
%%timeit -r 10 -n 1
exec_status = os.system("fpm run -- 1000000000 2>/dev/null")

After 1000000000 loops, Pi =         3.141592651590121
After 1000000000 loops, Pi =         3.141592651590121
After 1000000000 loops, Pi =         3.141592651590121
After 1000000000 loops, Pi =         3.141592651590121
After 1000000000 loops, Pi =         3.141592651590121
After 1000000000 loops, Pi =         3.141592651590121
After 1000000000 loops, Pi =         3.141592651590121
After 1000000000 loops, Pi =         3.141592651590121
After 1000000000 loops, Pi =         3.141592651590121
After 1000000000 loops, Pi =         3.141592651590121
2.29 s ± 120 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
